In [1]:
import os
import pandas as pd
from difflib import get_close_matches
import re
import json



In [2]:
# Define file paths (Update this to match your local file paths)
all_file_paths_original = [
    "median_weekly_income/2015_cpsaat39.xlsx",
    "median_weekly_income/2016_cpsaat39.xlsx",
    "median_weekly_income/2017_cpsaat39.xlsx",
    "median_weekly_income/2018_cpsaat39.xlsx",
    "median_weekly_income/2019_cpsaat39.xlsx",
    "median_weekly_income/2020_cpsaat39.xlsx",
    "median_weekly_income/2021_cpsaat39.xlsx",
    "median_weekly_income/2022_cpsaat39.xlsx",
    "median_weekly_income/2023_cpsaat39.xlsx",
    "median_weekly_income/2024_cpsaat39.xlsx",
]

In [3]:
# Define the occupation dictionary (Update if needed)
occupation_dict = {
    "Management, professional, and related occupations": {
        "Management, business, and financial operations occupations",
        "Management occupations",
        "Business and financial operations occupations",
        "Professional and related occupations",
        "Computer and mathematical occupations",
        "Computer occupations all other",
        "Miscellaneous mathematical science occupations",
        "Other mathematical science occupations",
        "Architecture and engineering occupations",
        "Life, physical, and social science occupations",
        "Community and social service occupations",
        "Legal occupations",
        "Education, training, and library occupations",
        "Arts, design, entertainment, sports, and media occupations",
        "Healthcare practitioners and technical occupations",
        "Other healthcare practitioners and technical occupations"
    },
    "Service occupations": {
        "Healthcare support occupations",
        "Miscellaneous healthcare support occupations including medical equipment preparers",
        "Protective service occupations",
        "Food preparation and serving related occupations",
        "Building and grounds cleaning and maintenance occupations",
        "Personal care and service occupations",
        "Service occupations"
    },
    "Sales and office occupations": {
        "Sales and related occupations",
        "Sales and office occupations",
        "Office and administrative support occupations"
    },
    "Production, transportation, and material moving occupations": {
        "Production occupations",
        "Production transportation and material moving occupations",
        "Transportation and material moving occupations"
    },
    "Natural resources, construction, and maintenance occupations": { 
        "Farming, fishing, and forestry occupations",
        "Construction and extraction occupations",
        "Installation, maintenance, and repair occupations",
        "Natural resources construction and maintenance occupations"
    }
}

In [4]:
# Define function to standardize occupation names for consistent comparison
def standardize_occupation(name: str) -> str:
    if pd.isna(name) or name is None:
        return None
    return name.strip().lower().replace(",", "").replace("-", " ").replace("_", " ")

Findings: Differences in Broader Occupation Categories Between 2015–2019 and 2020–2024

The only two broad occupation categories that existed from 2015–2019 but are missing from 2020–2024 are:
	1.	“miscellaneous mathematical science occupations”
	2.	“miscellaneous healthcare support occupations including medical equipment preparers”

✅ All other broader occupation categories remained consistent across both time periods.

### find the occupation dictionary

In [5]:
# Define output directory
output_dir_1 = "1_updated_median_weekly_income"
os.makedirs(output_dir_1, exist_ok=True)

# Define output files
output_categories_path = os.path.join(output_dir_1, "major_secondary_categories.xlsx")
output_dict_path = os.path.join(output_dir_1, "categories_dictionary.json")


# Create a lookup dictionary for secondary categories
secondary_category_lookup = {}
for main_category, subcategories in occupation_dict.items():
    for subcategory in subcategories:
        secondary_category_lookup[standardize_occupation(subcategory)] = main_category

# Dictionary to store unique categories across all years
comprehensive_categories = {"Major Categories": set(), "Secondary Categories": set()}

# List to store processed categories for all years
categories_list = []

# Process each file
for file_path in all_file_paths_original:
    try:
        # Extract the year from the filename
        year = os.path.basename(file_path).split("_")[0]

        # Load the Excel file
        df = pd.read_excel(file_path, dtype=str)

        # Trim whitespace from column names
        df.columns = df.columns.str.strip()

        # Identify the correct "Occupation" column dynamically
        occupation_col = next((col for col in df.columns if "occupation" in col.lower().strip()), None)
        if occupation_col is None:
            raise KeyError(f"No 'Occupation' column found in {file_path}.")

        # Rename column for consistency
        df = df.rename(columns={occupation_col: "Occupation title"})

        # Standardize occupation names for comparison
        df["Occupation title"] = df["Occupation title"].apply(standardize_occupation)

        # Add new columns for categories
        df["Main Category"] = None
        df["Secondary Category"] = None

        # Track current categories
        current_main_category = None
        current_secondary_category = None

        # Iterate through rows to assign categories
        for index, occupation in df["Occupation title"].fillna("").items():
            if occupation == "":  # Skip empty rows but don't reset previous valid categories
                continue
            elif occupation in occupation_dict:  # If it's a main category
                current_main_category = occupation
                current_secondary_category = None  # Reset secondary category
                comprehensive_categories["Major Categories"].add(occupation)
            elif occupation in secondary_category_lookup:  # If it's a secondary category
                if current_main_category and secondary_category_lookup[occupation] == current_main_category:
                    current_secondary_category = occupation
                    comprehensive_categories["Secondary Categories"].add(occupation)
            else:
                # If it's a specific occupation and no secondary category is assigned, keep the last assigned secondary category
                pass  

            # Store results
            categories_list.append({"Year": year, "Major Category": current_main_category, "Secondary Category": current_secondary_category})

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

# Convert results into a dataframe and save
categories_df = pd.DataFrame(categories_list)
categories_df.to_excel(output_categories_path, index=False)
print(f"✅ Categories file saved: {output_categories_path}")

# Convert comprehensive categories dictionary to JSON format for better readability
comprehensive_categories_dict = {
    "Major Categories": list(comprehensive_categories["Major Categories"]),
    "Secondary Categories": list(comprehensive_categories["Secondary Categories"]),
}

# Save as a JSON file
with open(output_dict_path, "w") as json_file:
    json.dump(comprehensive_categories_dict, json_file, indent=4)

print(f"✅ Dictionary file saved: {output_dict_path}")

print("\n🎉 All files processed successfully!")

✅ Categories file saved: 1_updated_median_weekly_income/major_secondary_categories.xlsx
✅ Dictionary file saved: 1_updated_median_weekly_income/categories_dictionary.json

🎉 All files processed successfully!


# add main category and secondary category

In [6]:
# Define output directory
output_dir = "1_updated_median_weekly_income"
os.makedirs(output_dir, exist_ok=True)  


# Process each file
for file_path in all_file_paths_original:
    try:
        # Load the Excel file
        df = pd.read_excel(file_path, dtype=str)

        # Trim whitespace from column names
        df.columns = df.columns.str.strip()

        # Identify the correct "Occupation" column dynamically
        occupation_col = None
        for col in df.columns:
            if "occupation" in col.lower():  # Case insensitive search
                occupation_col = col
                break

        if occupation_col is None:
            raise KeyError("No 'Occupation' column found.")

        # Rename column for consistency
        df = df.rename(columns={occupation_col: "Occupation title"})

        # Add new columns for categories
        df["Main Category"] = None
        df["Secondary Category"] = None

        # Track current main and secondary categories
        current_main_category = None
        current_secondary_category = None

        # Iterate through rows to assign main and secondary categories
        for index, occupation in df["Occupation title"].fillna("").str.strip().items():  # 🔹 FIXED: `.items()` instead of `.iteritems()`
            if occupation == "":  # Empty row resets the categories
                current_main_category = None
                current_secondary_category = None
            elif occupation in occupation_dict:  # Main category match
                current_main_category = occupation
                current_secondary_category = None
            elif any(occupation in subcategories for subcategories in occupation_dict.values()):  # Secondary category match
                current_secondary_category = occupation

            # Assign values using .loc[] for better performance
            df.loc[index, "Main Category"] = current_main_category
            df.loc[index, "Secondary Category"] = current_secondary_category

        # Generate output file path
        output_file_path = os.path.join(output_dir, os.path.basename(file_path))

        # Save the updated dataframe
        df.to_excel(output_file_path, index=False)

        print(f"✅ Processed and saved: {output_file_path}")

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

print("\n🎉 All files processed and saved in 'updated_median_weekly_income'!")

✅ Processed and saved: 1_updated_median_weekly_income/2015_cpsaat39.xlsx
✅ Processed and saved: 1_updated_median_weekly_income/2016_cpsaat39.xlsx
✅ Processed and saved: 1_updated_median_weekly_income/2017_cpsaat39.xlsx
✅ Processed and saved: 1_updated_median_weekly_income/2018_cpsaat39.xlsx
✅ Processed and saved: 1_updated_median_weekly_income/2019_cpsaat39.xlsx
✅ Processed and saved: 1_updated_median_weekly_income/2020_cpsaat39.xlsx
✅ Processed and saved: 1_updated_median_weekly_income/2021_cpsaat39.xlsx
✅ Processed and saved: 1_updated_median_weekly_income/2022_cpsaat39.xlsx
✅ Processed and saved: 1_updated_median_weekly_income/2023_cpsaat39.xlsx
✅ Processed and saved: 1_updated_median_weekly_income/2024_cpsaat39.xlsx

🎉 All files processed and saved in 'updated_median_weekly_income'!


In [7]:
# Function to standardize occupation names
def standardize_occupation_name(name: str) -> str:
    if pd.isna(name) or name is None:
        return None
    name = name.strip().lower().replace("-", " ").replace("_", " ")
    return re.sub(r"\s+", " ", name)  # Replace multiple spaces with a single space

# Function to standardize column names
def clean_column_name(name):
    if pd.isna(name) or name is None:
        return None
    name = name.strip().lower()
    name = re.sub(r"[\n\t]+", " ", name)  # Remove newlines and tabs
    name = re.sub(r"\s+", " ", name)  # Replace multiple spaces with a single space
    name = name.replace(" ", "_")  # Replace spaces with underscores
    return name

# Define directories
income_dir = "1_updated_median_weekly_income"
demographics_dir = "demographic_factors"
output_dir_2 = "2_updated_median_weekly_income"
os.makedirs(output_dir_2, exist_ok=True)

# Manual mapping for known occupation mismatches
manual_occupation_mapping = {
    "total 16 years and over": "total_population_race",
    "total employed": "total_population_age",
    "total full time wage and salary workers": "total_population_income"
}

# Get list of income files
income_files = sorted([f for f in os.listdir(income_dir) if f.endswith(".xlsx")])

# Process each year's data
for income_file in income_files:
    year = income_file.split("_")[0]  # Extract year from filename
    income_path = os.path.join(income_dir, income_file)

    # Load income data
    income_df = pd.read_excel(income_path, dtype=str)
    income_df.rename(columns=lambda x: clean_column_name(x.strip()), inplace=True)

    if "occupation_title" not in income_df.columns:
        print(f"❌ 'Occupation title' column not found in income file: {income_file}")
        continue
    
    income_df["occupation_title"] = income_df["occupation_title"].apply(standardize_occupation_name)
    
    # Load corresponding demographic files
    age_file = f"{year}_age.xlsx"
    race_file = f"{year}_race.xlsx"
    age_path = os.path.join(demographics_dir, age_file)
    race_path = os.path.join(demographics_dir, race_file)
    
    if os.path.exists(age_path) and os.path.exists(race_path):
        age_df = pd.read_excel(age_path, dtype=str)
        race_df = pd.read_excel(race_path, dtype=str)
        
        age_df.rename(columns=lambda x: clean_column_name(x.strip()), inplace=True)
        race_df.rename(columns=lambda x: clean_column_name(x.strip()), inplace=True)
        
        if "occupations" in race_df.columns:
            race_df.rename(columns={"occupations": "occupation_title"}, inplace=True)
        if "occupations" in age_df.columns:
            age_df.rename(columns={"occupations": "occupation_title"}, inplace=True)
        
        if "occupation_title" not in race_df.columns or "occupation_title" not in age_df.columns:
            print(f"❌ 'Occupation title' column not found in race or age files for {year}")
            continue
        
        # Standardize occupation names across datasets
        age_df["occupation_title"] = age_df["occupation_title"].apply(standardize_occupation_name)
        race_df["occupation_title"] = race_df["occupation_title"].apply(standardize_occupation_name)
        
        # Unique occupations before merging
        income_occ = set(income_df["occupation_title"].dropna())
        age_occ = set(age_df["occupation_title"].dropna())
        race_occ = set(race_df["occupation_title"].dropna())
        
        print(f"📊 {year} - Unique Occupations in Income: {len(income_occ)}")
        print(f"📊 {year} - Unique Occupations in Age: {len(age_occ)}")
        print(f"📊 {year} - Unique Occupations in Race: {len(race_occ)}")
        
        # Find missing occupations
        missing_in_income = (age_occ | race_occ) - income_occ
        if missing_in_income:
            print(f"⚠️ {year} - Missing Occupations in Income Data: {len(missing_in_income)}")
            
            # Rename columns in age_df and race_df except 'occupation_title'
            age_df.rename(columns={col: col + "_age" for col in age_df.columns if col != "occupation_title"}, inplace=True)
            race_df.rename(columns={col: col + "_race" for col in race_df.columns if col != "occupation_title"}, inplace=True)

            # Now merge datasets
            merged_df = income_df.merge(age_df, on="occupation_title", how="outer")
            merged_df = merged_df.merge(race_df, on="occupation_title", how="outer")
        
        print(f"✅ {year} - Final Merged Rows: {len(merged_df)}")
        
        # Save merged file
        output_path = os.path.join(output_dir_2, f"{year}_merged.xlsx")
        merged_df.to_excel(output_path, index=False)
        print(f"✅ Merged and saved: {output_path}")
    else:
        print(f"❌ Missing demographic files for {year}: {age_file}, {race_file}")

print("🎉 All available files processed and saved in '2_updated_median_weekly_income'!")

📊 2015 - Unique Occupations in Income: 565
📊 2015 - Unique Occupations in Age: 565
📊 2015 - Unique Occupations in Race: 565
⚠️ 2015 - Missing Occupations in Income Data: 2
✅ 2015 - Final Merged Rows: 667
✅ Merged and saved: 2_updated_median_weekly_income/2015_merged.xlsx
📊 2016 - Unique Occupations in Income: 565
📊 2016 - Unique Occupations in Age: 565
📊 2016 - Unique Occupations in Race: 565
⚠️ 2016 - Missing Occupations in Income Data: 2
✅ 2016 - Final Merged Rows: 667
✅ Merged and saved: 2_updated_median_weekly_income/2016_merged.xlsx
📊 2017 - Unique Occupations in Income: 565
📊 2017 - Unique Occupations in Age: 565
📊 2017 - Unique Occupations in Race: 565
⚠️ 2017 - Missing Occupations in Income Data: 2
✅ 2017 - Final Merged Rows: 667
✅ Merged and saved: 2_updated_median_weekly_income/2017_merged.xlsx
📊 2018 - Unique Occupations in Income: 565
📊 2018 - Unique Occupations in Age: 565
📊 2018 - Unique Occupations in Race: 565
⚠️ 2018 - Missing Occupations in Income Data: 2
✅ 2018 - Fin